# Creative Extension Analysis

In this notebook, we propose a creative extension analysis of the paper _Friendship and Mobility: User Movement in Location-Based Social Networks_. We chose to answer multiple research questions by using the same datasets (check-ins) and an additional one (airports). 

In particular, we aim to find out:
- Which countries travel the most long distance by plane?
- Where do people from different countries travel to the most?
- Where users' friends are based?
- Is it possible to predict user home areas based on their long distance travel patterns?

## Tools

In [51]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from numba import njit
import itertools

import reverse_geocoder
import pickle

## Collect Data

We load the check-ins datasets and the new airport dataset.

In [21]:
checkins_b = pd.read_csv("data/Brightkite_totalCheckins.txt", delimiter='\t', names=["user", "checkin_time", "latitude", "longitude", "location_id"])
checkins_g = pd.read_csv("data/Gowalla_totalCheckins.txt", delimiter='\t', names=["user", "checkin_time", "latitude", "longitude", "location_id"])

In [22]:
airports = pd.read_csv("data/airports.csv")

In the following cells, we will only display dataframes for Brightkite.

In [23]:
checkins_b.head()

,user,checkin_time,latitude,longitude,location_id
0,0,2010-10-17T01:48:53Z,39.747652,-104.992510,88c46bf20db295831bd2d1718ad7e6f5
1,0,2010-10-16T06:02:04Z,39.891383,-105.070814,7a0f88982aa015062b95e3b4843f9ca2
2,0,2010-10-16T03:48:54Z,39.891077,-105.068532,dd7cd3d264c2d063832db506fba8bf79
3,0,2010-10-14T18:25:51Z,39.750469,-104.999073,9848afcc62e500a01cf6fbf24b797732f8963683
4,0,2010-10-14T00:21:47Z,39.752713,-104.996337,2ef143e12038c870038df53e0478cefc


Here's the airport dataset.

In [24]:
airports.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR


In [25]:
print("Brightkite checkins dataset have {} rows".format(len(checkins_b)))
print("Gowalla checkins dataset have {} rows".format(len(checkins_g)))

Brightkite checkins dataset have 4747287 rows
Gowalla checkins dataset have 6442892 rows


## Work on Checkins Datasets

We check for missing values in both datasets

In [26]:
print(checkins_b.isna().sum())
print()
print(checkins_g.isna().sum())

user            0
checkin_time    6
latitude        6
longitude       6
location_id     6
dtype: int64

user            0
checkin_time    0
latitude        0
longitude       0
location_id     0
dtype: int64


In [27]:
checkins_b = checkins_b.dropna()

In [28]:
checkins_b["checkin_time"] = pd.to_datetime(checkins_b["checkin_time"], format = "%Y-%m-%dT%H:%M:%SZ")
checkins_g["checkin_time"] = pd.to_datetime(checkins_g["checkin_time"], format = "%Y-%m-%dT%H:%M:%SZ")

We need to check also if coordinates are correct values

In [29]:
print(len(checkins_b[((checkins_b["longitude"] < -180) | (checkins_b["longitude"] > 180)) | ((checkins_b["latitude"] < -90) | (checkins_b["latitude"] > 90))]))
print(len(checkins_g[((checkins_g["longitude"] < -180) | (checkins_g["longitude"] > 180)) | ((checkins_g["latitude"] < -90) | (checkins_g["latitude"] > 90))]))

109
29


We have 109 incorrect position values in the Brightkite dataset and 29 in Gowalla.

In [31]:
to_remove_b = checkins_b[((checkins_b["longitude"] < -180) | (checkins_b["longitude"] > 180)) | ((checkins_b["latitude"] < -90) | (checkins_b["latitude"] > 90))]
to_remove_g = checkins_g[((checkins_g["longitude"] < -180) | (checkins_g["longitude"] > 180)) | ((checkins_g["latitude"] < -90) | (checkins_g["latitude"] > 90))]

checkins_b = checkins_b.drop(to_remove_b.index)
checkins_g = checkins_g.drop(to_remove_g.index)

In [32]:
checkins_b.head()

,user,checkin_time,latitude,longitude,location_id
0,0,2010-10-17 01:48:53,39.747652,-104.992510,88c46bf20db295831bd2d1718ad7e6f5
1,0,2010-10-16 06:02:04,39.891383,-105.070814,7a0f88982aa015062b95e3b4843f9ca2
2,0,2010-10-16 03:48:54,39.891077,-105.068532,dd7cd3d264c2d063832db506fba8bf79
3,0,2010-10-14 18:25:51,39.750469,-104.999073,9848afcc62e500a01cf6fbf24b797732f8963683
4,0,2010-10-14 00:21:47,39.752713,-104.996337,2ef143e12038c870038df53e0478cefc


Also, the ```location_id``` column is useless.

In [33]:
checkins_b = checkins_b.drop(columns=['location_id'])
checkins_g = checkins_g.drop(columns=['location_id'])

In [34]:
checkins_b.head()

,user,checkin_time,latitude,longitude
0,0,2010-10-17 01:48:53,39.747652,-104.992510
1,0,2010-10-16 06:02:04,39.891383,-105.070814
2,0,2010-10-16 03:48:54,39.891077,-105.068532
3,0,2010-10-14 18:25:51,39.750469,-104.999073
4,0,2010-10-14 00:21:47,39.752713,-104.996337


## Work on Airport Dataset

## Exploratory Data Analysis

## Research Questions

### Which countries travel the most long distance by plane?

In [37]:
copy_checkins_b = checkins_b.copy()
copy_checkins_g = checkins_g.copy()

In [38]:
"""
Assign a cell_number based on the cantor pairing function and discretization into 25km * 25km cells.
"""
@njit
def assign_cell(lat, lon):
    lon_km = 111.320 * np.cos(np.deg2rad(lat)) * lon
    lat_km = 110.574 * lat
    #assign to intervals using the cantor pair function
    lat_km, lon_km = lat_km // 25, lon_km // 25 #now lat_km and lon_km contains the quotient from the division by 25.
    return (1/2)*(lat_km + lon_km)*(lat_km + lon_km + 1) + lon_km

For each checkin, we want to assign a cell_number based on the division of the world in 25km by 25km as in the replication. We do that using the cantor pairing function in the function above.

In [39]:
copy_checkins_b["cell_number"] = assign_cell(copy_checkins_b["latitude"].values, copy_checkins_b["longitude"].values)
copy_checkins_g["cell_number"] = assign_cell(copy_checkins_g["latitude"].values, copy_checkins_g["longitude"].values)

copy_checkins_b["cell_number"] = copy_checkins_b["cell_number"].astype('int')
copy_checkins_g["cell_number"] = copy_checkins_g["cell_number"].astype('int')

Next, we want to simplify the notion of home for each user because we only want to retrieve the country he lives in. To do that, we will simply take the average position of every checkin in each cell.

In [40]:
center_cells_b = copy_checkins_b[['latitude', 'longitude', 'cell_number']].groupby('cell_number').mean()
center_cells_g = copy_checkins_g[['latitude', 'longitude', 'cell_number']].groupby('cell_number').mean()

center_cells_b = center_cells_b.reset_index()
center_cells_g = center_cells_g.reset_index()

In [41]:
center_cells_b.head()

,cell_number,latitude,longitude
0,-286,64.545703,-149.087119
1,-284,64.152500,-145.842222
2,-282,63.562699,-142.300615
3,-280,63.934833,-145.788211
4,-279,63.661389,-144.064444


Now we work on the home for each user. Home is now the average position of checkins in each cell with the most checkins.

In [43]:
user_cell_b = copy_checkins_b.groupby(["user", "cell_number"]).count()
user_cell_g = copy_checkins_g.groupby(["user", "cell_number"]).count()

#We can remove these columns
user_cell_b = user_cell_b.drop(columns = ["latitude", "longitude"]) 
user_cell_g = user_cell_g.drop(columns = ["latitude", "longitude"]) 

#rename the column to count
user_cell_b.columns = ["count"] 
user_cell_g.columns = ["count"] 

#Sort by user to have user 0 first
user_cell_b = user_cell_b.sort_values("count").groupby(level=0).tail(1).sort_values('user') 
user_cell_g = user_cell_g.sort_values("count").groupby(level=0).tail(1).sort_values('user') 

#Reset the index to break the multiindex and keep only the column cell_number
user_cell_b = user_cell_b.reset_index().drop(columns=["count"])
user_cell_g = user_cell_g.reset_index().drop(columns=["count"]) 

In [44]:
user_cell_b.head()

,user,cell_number
0,0,16660
1,1,34813
2,2,16660
3,3,34813
4,4,51736


Now we can merge the two to have the home for each user as the average position of checkins in each cell.

In [47]:
homes_b = user_cell_b.merge(center_cells_b, on='cell_number')
homes_g = user_cell_g.merge(center_cells_g, on='cell_number')

In [48]:
homes_b.head()

,user,cell_number,latitude,longitude
0,0,16660,39.731991,-104.980205
1,2,16660,39.731991,-104.980205
2,8,16660,39.731991,-104.980205
3,12,16660,39.731991,-104.980205
4,13,16660,39.731991,-104.980205


Now for each home, we want to retrieve the country based on its coordinates.

In [26]:
homes_countries_b = homes_b.copy()
homes_countries_g = homes_g.copy()

In [27]:
homes_countries_b = homes_countries_b[['latitude', 'longitude']].groupby(['latitude', 'longitude']).count().reset_index()
homes_countries_g = homes_countries_g[['latitude', 'longitude']].groupby(['latitude', 'longitude']).count().reset_index()

In [28]:
homes_countries_b.head()

,latitude,longitude
0,-45.866346,170.516186
1,-45.076905,170.982070
2,-44.733333,170.466667
3,-43.576341,172.628973
4,-43.520079,172.604997


We create a function which will return the country code given a point coordinates.

In [50]:
"""
Return the ISO country code of a point in space

Arguments
---------
    -lat, lon: Latitude and Longitude of the point

Returns
---------
    -Returns the ISO country code of the point
"""


def retrieve_country(lat, lon):
    return reverse_geocoder.search((lat, lon))[0]['cc']

#vectorize the function
retrieve_country_vec = np.vectorize(retrieve_country)

The following cell takes a lot of time, so we pickled it.

In [54]:
# homes_countries_b['country'] = retrieve_country_vec(homes_countries_b['latitude'], 
#                                                     homes_countries_b['longitude'])

# homes_countries_g['country'] = retrieve_country_vec(homes_countries_g['latitude'], 
#                                                     homes_countries_g['longitude'])

In [30]:
#homes_countries_b.to_pickle('homes_countries_b.pkl')
#homes_countries_g.to_pickle('homes_countries_g.pkl')

In [54]:
homes_countries_b = pd.read_pickle("data/homes_countries_b.pkl")
homes_countries_g = pd.read_pickle("data/homes_countries_g.pkl")

In [57]:
homes_countries_b.head()

,latitude,longitude,country
0,-45.866346,170.516186,NZ
1,-45.076905,170.982070,NZ
2,-44.733333,170.466667,NZ
3,-43.576341,172.628973,NZ
4,-43.520079,172.604997,NZ


Now that we have for each 'home' the country code, we can now merge the two datasets so that we have a complete dataframe.

In [58]:
homes_countries_b = homes_b.merge(homes_countries_b, on=['latitude', 'longitude'])
homes_countries_g = homes_g.merge(homes_countries_g, on=['latitude', 'longitude'])

In [60]:
homes_countries_b.head()

,user,cell_number,latitude,longitude,country
0,0,16660,39.731991,-104.980205,US
1,2,16660,39.731991,-104.980205,US
2,8,16660,39.731991,-104.980205,US
3,12,16660,39.731991,-104.980205,US
4,13,16660,39.731991,-104.980205,US


### Where do people from different countries travel to the most?

### Check if it is possible to predict user home areas based on their long distance travel patterns?